### Tweet Spam Or Ham
- In this below dataset we are trying to load our tweets with the label of 0 as Ham and 1 as Spam, and we will try to apply some data cleaning and we will bulid one tweet spam and ham classfier.

### Importing Packages

In [ ]:
import re
import pandas as pd

### Loading Dataset

In [3]:
df = pd.read_csv('train_tweet.csv')
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


### Information of attributes

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


### 

### Splitting into Features and Labels

In [5]:
df_features = df.drop(['id','label'],axis=1)
df_label = df[['label']]
print(df_features.info())
print(df_label.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   31962 non-null  object
dtypes: object(1)
memory usage: 249.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   label   31962 non-null  int64
dtypes: int64(1)
memory usage: 249.8 KB
None


### Checking for N-Unique values in label

In [6]:
print(df_label.nunique())
print(df_label.value_counts())

label    2
dtype: int64
label
0        29720
1         2242
dtype: int64


## Data Cleaning & Preprocessing
### Applying Regular expression on Feature column (Tweet)
- Tweet data cleaning.
    - @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.
    - @user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.
- We have cleaned dataset tweet column using regular expression, and applied on tweet column and we have stored that on cleaned_tweet column

In [7]:
def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove usernames
    text = re.sub(r'#', '', text)  # Remove hashtags
    text = re.sub(r'RT[\s]+', '', text)  # Remove retweets
    text = re.sub(r'https?:\/\/\S+', '', text)  # Remove URLs
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters (including emojis)
    return text

df_features['cleaned_tweet'] = df_features['tweet'].apply(clean_text)
print(df_features.head())

                                               tweet  \
0   @user when a father is dysfunctional and is s...   
1  @user @user thanks for #lyft credit i can't us...   
2                                bihday your majesty   
3  #model   i love u take with u all the time in ...   
4             factsguide: society now    #motivation   

                                       cleaned_tweet  
0  when a father is dysfunctional and is so selfi...  
1  thanks for lyft credit i can't use cause they ...  
2                                bihday your majesty  
3  model i love u take with u all the time in ur!...  
4                 factsguide: society now motivation  


### Feature Extraction (TF-IDF Vectorization)
- We are using vectorization to encode text to numerical dtype
- We have initialized TfidVectorized with max_features=2000 and stop_wrods='english', made an object as vectorizer and applied fit_transform() method on df_features['cleaned_tweet] as array.

In [8]:
vectorizer = TfidfVectorizer(max_features=2000,stop_words='english')
df_features_vectorized = vectorizer.fit_transform(df_features['cleaned_tweet']).toarray()

### Handling Imbalanced Data

In [9]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(df_features_vectorized,df_label)
print(len(X_resampled),len(y_resampled))
print(X_resampled.shape,y_resampled.shape)

59440 59440
(59440, 2000) (59440, 1)


### Train Test Split
-- Train test split we are passing two argument first is X variable as (df_features_vectorized) and second is (df_label), With transformation of Regular Expression and Vectorization on our feature column.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled,test_size=0.2,random_state=42)
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(47552, 2000) (11888, 2000)
(47552, 1) (11888, 1)
